# 0. Downloading and preparing the dataset

In [4]:
import pandas as pd

In [3]:
!gdown --id 15dseCx8KelNjgylcbw7Bx4MlUvf25m6A

Downloading...
From: https://drive.google.com/uc?id=15dseCx8KelNjgylcbw7Bx4MlUvf25m6A
To: /content/dimensionality_reduction_cbb.csv
100% 209k/209k [00:00<00:00, 77.8MB/s]


In [5]:
df = pd.read_csv('dimensionality_reduction_cbb.csv')

In [6]:
df.head()

,TEAM,CONF,G,W,ADJOE,ADJDE,BARTHAG,EFG_O,EFG_D,TOR,TORD,ORB,DRB,FTR,FTRD,2P_O,2P_D,3P_O,3P_D,ADJ_T,WAB,POSTSEASON,SEED,YEAR
0,North Carolina,ACC,40,33,123.3,94.9,0.9531,52.6,48.1,15.4,18.2,40.7,30.0,32.3,30.4,53.9,44.6,32.7,36.2,71.7,8.6,2ND,1.0,2016
1,Wisconsin,B10,40,36,129.1,93.6,0.9758,54.8,47.7,12.4,15.8,32.1,23.7,36.2,22.4,54.8,44.7,36.5,37.5,59.3,11.3,2ND,1.0,2015
2,Michigan,B10,40,33,114.4,90.4,0.9375,53.9,47.7,14.0,19.5,25.5,24.9,30.7,30.0,54.7,46.8,35.2,33.2,65.9,6.9,2ND,3.0,2018
3,Texas Tech,B12,38,31,115.2,85.2,0.9696,53.5,43.0,17.7,22.8,27.4,28.7,32.9,36.6,52.8,41.9,36.5,29.7,67.5,7.0,2ND,3.0,2019
4,Gonzaga,WCC,39,37,117.8,86.3,0.9728,56.6,41.1,16.2,17.1,30.0,26.2,39.0,26.9,56.3,40.0,38.2,29.0,71.5,7.7,2ND,1.0,2017


# 1. Missing Value Ratio Filter (MVRF)

In [ ]:
df = pd.read_csv('dimensionality_reduction_cbb.csv')

In [ ]:
df.head()

,TEAM,CONF,G,W,ADJOE,ADJDE,BARTHAG,EFG_O,EFG_D,TOR,TORD,ORB,DRB,FTR,FTRD,2P_O,2P_D,3P_O,3P_D,ADJ_T,WAB,POSTSEASON,SEED,YEAR
0,North Carolina,ACC,40,33,123.3,94.9,0.9531,52.6,48.1,15.4,18.2,40.7,30.0,32.3,30.4,53.9,44.6,32.7,36.2,71.7,8.6,2ND,1.0,2016
1,Wisconsin,B10,40,36,129.1,93.6,0.9758,54.8,47.7,12.4,15.8,32.1,23.7,36.2,22.4,54.8,44.7,36.5,37.5,59.3,11.3,2ND,1.0,2015
2,Michigan,B10,40,33,114.4,90.4,0.9375,53.9,47.7,14.0,19.5,25.5,24.9,30.7,30.0,54.7,46.8,35.2,33.2,65.9,6.9,2ND,3.0,2018
3,Texas Tech,B12,38,31,115.2,85.2,0.9696,53.5,43.0,17.7,22.8,27.4,28.7,32.9,36.6,52.8,41.9,36.5,29.7,67.5,7.0,2ND,3.0,2019
4,Gonzaga,WCC,39,37,117.8,86.3,0.9728,56.6,41.1,16.2,17.1,30.0,26.2,39.0,26.9,56.3,40.0,38.2,29.0,71.5,7.7,2ND,1.0,2017


In [20]:
df.drop(df.columns[df.isna().sum() / len(df) > 0.8], axis=1)

,TEAM,CONF,G,W,ADJOE,ADJDE,BARTHAG,EFG_O,EFG_D,TOR,TORD,ORB,DRB,FTR,FTRD,2P_O,2P_D,3P_O,3P_D,ADJ_T,WAB,YEAR
0,North Carolina,ACC,40,33,123.3,94.9,0.9531,52.6,48.1,15.4,18.2,40.7,30.0,32.3,30.4,53.9,44.6,32.7,36.2,71.7,8.6,2016
1,Wisconsin,B10,40,36,129.1,93.6,0.9758,54.8,47.7,12.4,15.8,32.1,23.7,36.2,22.4,54.8,44.7,36.5,37.5,59.3,11.3,2015
2,Michigan,B10,40,33,114.4,90.4,0.9375,53.9,47.7,14.0,19.5,25.5,24.9,30.7,30.0,54.7,46.8,35.2,33.2,65.9,6.9,2018
3,Texas Tech,B12,38,31,115.2,85.2,0.9696,53.5,43.0,17.7,22.8,27.4,28.7,32.9,36.6,52.8,41.9,36.5,29.7,67.5,7.0,2019
4,Gonzaga,WCC,39,37,117.8,86.3,0.9728,56.6,41.1,16.2,17.1,30.0,26.2,39.0,26.9,56.3,40.0,38.2,29.0,71.5,7.7,2017
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1752,Texas A&M,SEC,35,22,111.2,94.7,0.8640,51.4,46.9,19.2,15.3,33.9,27.3,32.0,27.6,52.5,45.7,32.9,32.6,70.3,1.9,2018
1753,LSU,SEC,35,28,117.9,96.6,0.9081,51.2,49.9,17.9,20.1,36.7,30.8,37.1,33.1,52.9,49.4,31.9,33.7,71.2,7.3,2019
1754,Tennessee,SEC,36,31,122.8,95.2,0.9488,55.3,48.1,15.8,18.0,31.6,30.2,33.3,34.9,55.4,44.7,36.7,35.4,68.8,9.9,2019
1755,Gonzaga,WCC,35,27,117.4,94.5,0.9238,55.2,44.8,17.1,15.1,32.1,26.0,34.4,28.1,54.3,44.4,37.8,30.3,68.2,2.1,2016


# 2. Low Variance Filter (LVF)

In [21]:
df = pd.read_csv('dimensionality_reduction_cbb.csv')

In [26]:
df2 = df.fillna(df.mode().loc[0])

In [51]:
low_var = (df2.var() < 10)
cols_to_del = list(low_var[low_var].index)

In [53]:
df2.drop(cols_to_del, axis=1, inplace=True)

In [54]:
df2

,TEAM,CONF,W,ADJOE,ADJDE,ORB,FTR,FTRD,2P_O,2P_D,ADJ_T,WAB,POSTSEASON
0,North Carolina,ACC,33,123.3,94.9,40.7,32.3,30.4,53.9,44.6,71.7,8.6,2ND
1,Wisconsin,B10,36,129.1,93.6,32.1,36.2,22.4,54.8,44.7,59.3,11.3,2ND
2,Michigan,B10,33,114.4,90.4,25.5,30.7,30.0,54.7,46.8,65.9,6.9,2ND
3,Texas Tech,B12,31,115.2,85.2,27.4,32.9,36.6,52.8,41.9,67.5,7.0,2ND
4,Gonzaga,WCC,37,117.8,86.3,30.0,39.0,26.9,56.3,40.0,71.5,7.7,2ND
...,...,...,...,...,...,...,...,...,...,...,...,...,...
1752,Texas A&M,SEC,22,111.2,94.7,33.9,32.0,27.6,52.5,45.7,70.3,1.9,S16
1753,LSU,SEC,28,117.9,96.6,36.7,37.1,33.1,52.9,49.4,71.2,7.3,S16
1754,Tennessee,SEC,31,122.8,95.2,31.6,33.3,34.9,55.4,44.7,68.8,9.9,S16
1755,Gonzaga,WCC,27,117.4,94.5,32.1,34.4,28.1,54.3,44.4,68.2,2.1,S16


In [55]:
df2.var()

W        42.844496
ADJOE    53.362660
ADJDE    41.895531
ORB      16.824613
FTR      23.859306
FTRD     34.821035
2P_O     11.711017
2P_D     10.812684
ADJ_T    10.620558
WAB      48.841845
dtype: float64

# 3. High Correlation Filter (HCF)

In [56]:
df = pd.read_csv('dimensionality_reduction_cbb.csv')

In [81]:
corr_df = df.corr().abs().unstack().reset_index()

In [82]:
corr_df['high'] = corr_df[0] > 0.7

In [90]:
corr_df[corr_df['high'] & (corr_df['level_0'] != corr_df['level_1'])]\
       .sort_values(by=0,ascending=False)\
       .reset_index(drop=True)\
       .drop('high', axis=1)

,level_0,level_1,0
0,BARTHAG,WAB,0.941776
1,WAB,BARTHAG,0.941776
2,SEED,WAB,0.922111
3,WAB,SEED,0.922111
4,EFG_D,2P_D,0.907933
5,2P_D,EFG_D,0.907933
6,W,WAB,0.905029
7,WAB,W,0.905029
8,2P_O,EFG_O,0.893530
9,EFG_O,2P_O,0.893530
